In [ ]:
# 📦 Install required packages
%pip install langchain langchain-openai chromadb tiktoken python-dotenv

# 🔗 LangChain + ChromaDB + OpenAI RAG
This notebook uses LangChain's RAG workflow with OpenAI embeddings and Chroma.

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(api_key=openai_key, model="gpt-4")


In [ ]:
# Load and split documents
loader = DirectoryLoader(path="./data/new_articles/", glob="*.txt", loader_cls=TextLoader)
document = loader.load()

text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(document)
print(f"Number of documents: {len(documents)}")

In [ ]:
# Embedding and persistence
embedding = OpenAIEmbeddings(api_key=openai_key, model="text-embedding-3-small")
persist_directory = "./db/chroma_db_real_world"
vectordb = Chroma.from_documents(documents=documents, embedding=embedding, persist_directory=persist_directory)
retriever = vectordb.as_retriever()

In [ ]:
# RAG Prompt & Chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the answer concise.\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", "{input}")])
qa_chain = create_stuff_documents_chain(llm=model, prompt=prompt)
rag_chain = create_retrieval_chain(retriever, qa_chain)

In [ ]:
# Query
response = rag_chain.invoke({"input": "talk about databricks news"})
print(response["answer"])